In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files

files.upload()

Saving cleaned_data_5.csv to cleaned_data_5.csv


{'cleaned_data_5.csv': b'raw,processed,label\n"Kalau peta politik lihat siapa yang dulu bersama Pak Anis dan sekarang dengan Pak Prab, sama saja dan selama Pak Anis jadi gubernur, cek apakah Jakarta sudah menjadi Suriah atau tidak.",peta politik lihat pak anis pak prab pak anis jadi gubernur cek jakarta menjadi suriah,Geografi\nPENGGUNAAN Desa saya dekat pegunungan tinggi para petani menjual hasil bumi ke Riau palembang di daerah tersebut banyak jalan becek dan truk pulang lagi petani sakit tapi sebagian besar desa saya masih petik anies baswedan jadi yaaah gak sedih,penggunaan desa pegunungan tinggi petani menjual hasil bumi riau palembang daerah jalan becek truk pulang petani sakit sebagian besar desa petik anies baswedan jadi ya sedih,Geografi\nDilansir dari video yang diposting pada laman ANTARA calon presiden nomor urut dua ini menghadiri acara relawan Indonesia maju di Lapangan Temanggung Abdul Jamal Kota Batam #IndonesiaSentris #IndonesiaHijau #02Melanjutkan #AnakMudaIndonesiaEm

In [2]:
df = pd.read_csv('cleaned_data_5.csv')

In [3]:
df = df[['processed', 'label']]

In [4]:
train_df, test_df = train_test_split(df, shuffle=False, test_size=0.05)

In [5]:
df.head()

,processed,label
0,peta politik lihat pak anis pak prab pak anis ...,Geografi
1,penggunaan desa pegunungan tinggi petani menju...,Geografi
2,dilansir video diposting laman calon presiden ...,Politik
3,penggunaan pejabat militer kembali penjara unt...,PertahanandanKeamanan
4,anies takut dilabeli intoleran datang pernikah...,SumberDayaAlam


In [5]:

test_df.reset_index(drop=True, inplace=True)


In [7]:
test_df.head(300
             )

,processed,label
0,pasangan capres cawapres ri no ganjar pranowo ...,Ekonomi
1,calon presiden ri ganjar pranowo bersumpah ber...,Ideologi
2,penggunaan pilihlah HTTPURL prabowo memerlukan...,PertahanandanKeamanan
3,anis beswedan berkomitmen untuk memperbaiki ja...,SumberDayaAlam
4,baitul maqdis gaza penjaganya sakit pak prabow...,SosialBudaya
...,...,...
295,kapolri jenderal listyo sigit prabowo hadiri p...,SosialBudaya
296,gunakan penggunaan penggunaan,Geografi
297,pak anies menghargai orang lihat anak muda ber...,SosialBudaya
298,aksi anies berjanji membangun jalur produksi u...,SumberDayaAlam


In [8]:
train_df.head()

,processed,label
0,peta politik lihat pak anis pak prab pak anis ...,Geografi
1,penggunaan desa pegunungan tinggi petani menju...,Geografi
2,dilansir video diposting laman calon presiden ...,Politik
3,penggunaan pejabat militer kembali penjara unt...,PertahanandanKeamanan
4,anies takut dilabeli intoleran datang pernikah...,SumberDayaAlam


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")

def preprocess(data, tokenizer, max_len=60):
    inputs = tokenizer(
        data['processed'].tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return inputs

# Preprocess training data
train_inputs = preprocess(train_df, tokenizer)
train_labels = train_df['label'].astype('category').cat.codes  # Convert labels to numeric

# Preprocess testing data
test_inputs = preprocess(test_df, tokenizer)
test_labels = test_df['label'].astype('category').cat.codes  # Convert labels to numeric


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings, label):
        self.encodings = encodings
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.label[idx]).long()
        return item

    def __len__(self):
        return len(self.label)

train_dataset = CustomDataset(train_inputs, train_labels)
test_dataset = CustomDataset(test_inputs, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
from transformers import AutoModelForSequenceClassification, AdamW

model = AutoModelForSequenceClassification.from_pretrained("indolem/indobertweet-base-uncased", num_labels=len(train_df['label'].unique()))
optimizer = AdamW(model.parameters(), lr=1e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

model.train()
for epoch in range(15):
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


100%|██████████| 157/157 [00:49<00:00,  3.20it/s]


In [10]:
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []
for batch in test_dataloader:
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8745247148288974


In [11]:
from sklearn.metrics import balanced_accuracy_score

print(balanced_accuracy_score(true_labels, predictions))

0.8725672618461766


In [12]:
prediction_df = pd.read_csv('data_test.csv')

In [14]:
prediction_df = prediction_df.drop(columns=['raw'])


In [15]:
prediction_df

,processed
0,orang pro demokrasi negara punya kesempatan me...
1,prabowo ditanya soal hutang luar negeri menjaw...
2,kiki daliyo ganjar pranowo beliau sosok mengag...
3,kumparan prabowo gibran melakukan kesejahteraa...
4,sniperruben zulhendra ainunnajib menyambung ju...
...,...
995,bikin bangga deh ganjar mahfud alokasikan sete...
996,pak jokowi pilpres berbesar hati merangkul pak...
997,datuakrajoangek sebaiknya sih gemot usah ikuta...
998,kebiasaan merembuk bermusyawarah jadi gaya kep...


In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")

def preprocess(data, tokenizer, max_len=60):
    inputs = tokenizer(
        data['processed'].tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return inputs

In [17]:
prediction_inputs = preprocess(prediction_df, tokenizer)

In [18]:
class PredictionDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create dataset and DataLoader for new data
pred_dataset = PredictionDataset(prediction_inputs)
pred_dataloader = DataLoader(pred_dataset, batch_size=16, shuffle=False)

In [19]:
model.eval()

predictions = []
with torch.no_grad():
    for batch in pred_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(batch_predictions)


In [20]:
prediction_df['predictions'] = predictions

In [21]:
prediction_df


,processed,predictions
0,orang pro demokrasi negara punya kesempatan me...,5
1,prabowo ditanya soal hutang luar negeri menjaw...,5
2,kiki daliyo ganjar pranowo beliau sosok mengag...,3
3,kumparan prabowo gibran melakukan kesejahteraa...,5
4,sniperruben zulhendra ainunnajib menyambung ju...,3
...,...,...
995,bikin bangga deh ganjar mahfud alokasikan sete...,1
996,pak jokowi pilpres berbesar hati merangkul pak...,5
997,datuakrajoangek sebaiknya sih gemot usah ikuta...,5
998,kebiasaan merembuk bermusyawarah jadi gaya kep...,1


In [39]:
category_mapping = {0: 'Demografi',
 1: 'Ekonomi',
 2: 'Geografi',
 3: 'Ideologi',
 4: 'Pertahanan dan Keamanan',
 5: 'Politik',
 6: 'Sosial Budaya',
 7: 'Sumber Daya Alam'}

In [40]:
string_predictions = [category_mapping[pred] for pred in predictions]

In [41]:
string_predictions

['Politik',
 'Politik',
 'Ideologi',
 'Politik',
 'Ideologi',
 'Politik',
 'Politik',
 'Pertahanan dan Keamanan',
 'Ideologi',
 'Ekonomi',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Sosial Budaya',
 'Politik',
 'Politik',
 'Ekonomi',
 'Demografi',
 'Politik',
 'Politik',
 'Politik',
 'Ekonomi',
 'Sumber Daya Alam',
 'Ekonomi',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Ekonomi',
 'Politik',
 'Ekonomi',
 'Politik',
 'Ekonomi',
 'Ekonomi',
 'Ekonomi',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Ekonomi',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Pertahanan dan Keamanan',
 'Politik',
 'Politik',
 'Politik',
 'Ekonomi',
 'Politik',
 'Politik',
 'Politik',
 'Pertahanan dan Keamanan',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Politik',
 'Ekonomi',
 'Politik',
 'Ekonom

In [42]:
prediction_df['preds'] = string_predictions

In [43]:
prediction_df

,IDText,Kelas,preds
0,TXT0001,Politik,Politik
1,TXT0002,Politik,Politik
2,TXT0003,Ideologi,Ideologi
3,TXT0004,Politik,Politik
4,TXT0005,Ideologi,Ideologi
...,...,...,...
995,TXT0996,Ekonomi,Ekonomi
996,TXT0997,Politik,Politik
997,TXT0998,Politik,Politik
998,TXT0999,Ekonomi,Ekonomi


In [44]:
prediction_df.drop(['Kelas'], axis=1, inplace=True)

In [45]:
prediction_df

,IDText,preds
0,TXT0001,Politik
1,TXT0002,Politik
2,TXT0003,Ideologi
3,TXT0004,Politik
4,TXT0005,Ideologi
...,...,...
995,TXT0996,Ekonomi
996,TXT0997,Politik
997,TXT0998,Politik
998,TXT0999,Ekonomi


In [31]:
num_rows = len(prediction_df)
prediction_df['IDText'] = ['TXT' + str(i+1).zfill(4) for i in range(num_rows)]

In [32]:
prediction_df

,preds,IDText
0,Politik,TXT0001
1,Politik,TXT0002
2,Ideologi,TXT0003
3,Politik,TXT0004
4,Ideologi,TXT0005
...,...,...
995,Ekonomi,TXT0996
996,Politik,TXT0997
997,Politik,TXT0998
998,Ekonomi,TXT0999


In [33]:
prediction_df = prediction_df[['IDText', 'preds']]

In [46]:
prediction_df.rename(columns={'preds': 'Kelas'}, inplace=True)

In [47]:
prediction_df

,IDText,Kelas
0,TXT0001,Politik
1,TXT0002,Politik
2,TXT0003,Ideologi
3,TXT0004,Politik
4,TXT0005,Ideologi
...,...,...
995,TXT0996,Ekonomi
996,TXT0997,Politik
997,TXT0998,Politik
998,TXT0999,Ekonomi


In [48]:
prediction_df.to_csv('SD2024040000232.csv', index=False)